<a href="https://colab.research.google.com/github/abxda/COLMEX-ML/blob/main/Semana_09_CENSO_DENUE_COLMEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://drive.google.com/file/d/1LVPNaUxto31HE-UQIO1-4l6XPER9Ry5l/view?usp=sharing
#https://drive.google.com/file/d/1O_GKDVQSoBAEuHjCMHMkknDxSuZd3i_A/view?usp=sharing
# Instalar gdown si no está disponible
#!pip install gdown

In [ ]:
pip install pyarrow geopandas

In [ ]:

import duckdb
import pandas as pd
import geopandas as gpd
from shapely import wkb

In [ ]:
# Descargar el archivo usando el ID de Google Drive
!gdown --id 1LVPNaUxto31HE-UQIO1-4l6XPER9Ry5l

In [ ]:
!gdown --id 1O_GKDVQSoBAEuHjCMHMkknDxSuZd3i_A

In [ ]:
!ls -lh

In [ ]:
# Conectar a la base de datos
db_path = "/content/datos_censo_nacional_s9.duckdb"
conn = duckdb.connect(db_path)

# Obtener listado de tablas
tables = conn.execute("SHOW TABLES").fetchall()

# Generar reporte
reporte = {}
for table in tables:
    table_name = table[0]
    # Obtener metadatos de columnas
    columns = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
    # Almacenar nombre y tipo de cada columna
    reporte[table_name] = [(col[1], col[2]) for col in columns]

# Mostrar resultados en formato legible
for tabla, columnas in reporte.items():
    print(f"\n[ Tabla: {tabla} ]")
    print("--------------------")
    for columna in columnas:
        print(f"- {columna[0]} ({columna[1]})")
    print("--------------------")

conn.close()

In [ ]:
# Conectar a la base de datos
db_path = "/content/denue_total_4.duckdb"
conn = duckdb.connect(db_path)

# Obtener listado de tablas
tables = conn.execute("SHOW TABLES").fetchall()

# Generar reporte
reporte = {}
for table in tables:
    table_name = table[0]
    # Obtener metadatos de columnas
    columns = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
    # Almacenar nombre y tipo de cada columna
    reporte[table_name] = [(col[1], col[2]) for col in columns]

# Mostrar resultados en formato legible
for tabla, columnas in reporte.items():
    print(f"\n[ Tabla: {tabla} ]")
    print("--------------------")
    for columna in columnas:
        print(f"- {columna[0]} ({columna[1]})")
    print("--------------------")


# Verificar creación del nuevo archivo
print("\nBase de datos creada:")
!ls -lh {db_path}

conn.close()

In [ ]:
# Crear nueva base de datos
new_db_path = "/content/denue_total_est_per_ocu_final_5.duckdb"
conn = duckdb.connect(new_db_path)

# Cargar extensión espacial
conn.install_extension("spatial")
conn.load_extension("spatial")

# Crear tabla compatible
conn.execute(f"""
ATTACH DATABASE '/content/denue_total_4.duckdb' AS source_db;

CREATE TABLE denue_est_per_ocu AS
SELECT
    id,
    clee,
    SUBSTRING(codigo_act, 1, 2) AS codigo_act_2c,
    codigo_act,
    LOWER(nom_estab) AS nom_estab,
    cve_ent,
    LOWER(entidad) AS entidad,
    cve_mun,
    LOWER(municipio) AS municipio,
    cve_loc,
    LOWER(localidad) AS localidad,
    CASE
        WHEN per_ocu LIKE '%0 a 5 personas%' THEN 2.5
        WHEN per_ocu LIKE '%6 a 10 personas%' THEN 8
        WHEN per_ocu LIKE '%11 a 30 personas%' THEN 20.5
        WHEN per_ocu LIKE '%31 a 50 personas%' THEN 40.5
        WHEN per_ocu LIKE '%51 a 100 personas%' THEN 75.5
        WHEN per_ocu LIKE '%101 a 250 personas%' THEN 175.5
        WHEN per_ocu LIKE '%251 y más personas%' THEN 350
        ELSE 1.0
    END AS est_per_ocu,
    geometry
FROM source_db.denue;
""")


# Verificar la nueva tabla
result = conn.execute("PRAGMA table_info(denue_est_per_ocu)").fetchall()
print("\nEsquema de la nueva tabla:")
for col in result:
    print(f"- {col[1]} ({col[2]})")

# Verificar creación del nuevo archivo
print("\nBase de datos creada:")
!ls -lh {new_db_path}

# Verificar tablas en la nueva base de datos
tables = conn.execute("SHOW TABLES").fetchall()
print("\nTablas en la nueva base de datos:")
for table in tables:
    print(f"- {table[0]}")

conn.close()

In [ ]:

# Ruta al archivo DuckDB (la tabla 'nacional' se creará dentro de este archivo)
db_path = "/content/denue_total_est_per_ocu_final_5.duckdb"
conn = duckdb.connect(db_path)

# Instalar y cargar la extensión espacial (si aún no se hizo)
conn.execute("INSTALL spatial;")
conn.execute("LOAD spatial;")

# Crear la tabla 'nacional' directamente en el archivo DuckDB
conn.execute("""
DROP TABLE IF EXISTS nacional;
CREATE TABLE nacional AS
WITH
  -- Extraer OXXOs
  oxxos AS (
    SELECT
      1 AS klass,
      id,
      codigo_act,
      codigo_act_2c,
      clee,
      LOWER(nom_estab) AS nom_estab,
      cve_ent,
      est_per_ocu as personal,
      geometry
    FROM denue_est_per_ocu
    WHERE codigo_act = '462112'
      AND lower(nom_estab) LIKE '%oxxo%'
      AND lower(nom_estab) NOT LIKE '%distribuc%'
  ),
  -- Contar el número de OXXOs
  num_oxxos AS (
    SELECT count(*) AS n FROM oxxos
  ),
  -- Extraer tiendas de abarrotes
  abarrotes AS (
    SELECT
      0 AS klass,
      id,
      codigo_act,
      codigo_act_2c,
      clee,
      LOWER(nom_estab) AS nom_estab,
      cve_ent,
      est_per_ocu as personal,
      geometry
    FROM denue_est_per_ocu
    WHERE codigo_act = '461110'
  ),
  -- Seleccionar aleatoriamente tantas tiendas de abarrotes como OXXOs existen
  abarrotes_sample AS (
    SELECT *
    FROM abarrotes
    ORDER BY random()
    LIMIT (SELECT n FROM num_oxxos)
  )
-- Unir ambos conjuntos y mezclarlos aleatoriamente
SELECT *
FROM oxxos
UNION ALL
SELECT *
FROM abarrotes_sample;
""")
conn.close()


In [ ]:
# Ruta al archivo DuckDB (la tabla 'nacional' se creará dentro de este archivo)
db_path = "/content/denue_total_est_per_ocu_final_5.duckdb"
conn = duckdb.connect(db_path)

# Instalar y cargar la extensión espacial (si aún no se hizo)
conn.execute("INSTALL spatial;")
conn.execute("LOAD spatial;")
# Mostrar una muestra de algunos registros de la tabla 'nacional'
sample = conn.execute("SELECT * FROM denue_est_per_ocu order by random() LIMIT 100").fetch_df()

conn.close()
sample

In [ ]:
# Conectar a las bases de datos y configurar extensiones
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')
conn.execute("INSTALL spatial; LOAD spatial;")

In [ ]:
# 3. Crear la tabla 'nacional_buffer_100' usando el buffer de 0.000898 grados (aproximadamente 100 m)
conn.execute("""
DROP TABLE IF EXISTS nacional_buffer_100;
CREATE TABLE nacional_buffer_100 AS
SELECT *,
       ST_Buffer(geometry, 0.000898) AS buffer_100
FROM nacional;
""")

# 4. Crear la tabla 'nacional_buffer_500' usando el buffer de 0.00449 grados (aproximadamente 500 m)
conn.execute("""
DROP TABLE IF EXISTS nacional_buffer_500;
CREATE TABLE nacional_buffer_500 AS
SELECT *,
       ST_Buffer(geometry, 0.00449) AS buffer_500
FROM nacional;
""")

# 5. Agregar (sumar) est_per_ocu para los puntos DENUE dentro del buffer de 100 metros
conn.execute("""
DROP TABLE IF EXISTS agg_100;
CREATE TABLE agg_100 AS
SELECT
    n.id AS nacional_id,
    SUM(d.est_per_ocu) AS epo_100
FROM nacional_buffer_100 n
JOIN denue_est_per_ocu d ON ST_Intersects(n.buffer_100, d.geometry)
GROUP BY n.id;
""")

# 6. Agregar (sumar) est_per_ocu para los puntos DENUE dentro del buffer de 500 metros
conn.execute("""
DROP TABLE IF EXISTS agg_500;
CREATE TABLE agg_500 AS
SELECT
    n.id AS nacional_id,
    SUM(d.est_per_ocu) AS epo_500
FROM nacional_buffer_500 n
JOIN denue_est_per_ocu d ON ST_Intersects(n.buffer_500, d.geometry)
GROUP BY n.id;
""")

# 7. Crear la tabla final 'nacional_final' uniendo la tabla 'nacional_proj' con las sumas agregadas
conn.execute("""
DROP TABLE IF EXISTS nacional_final;
CREATE TABLE nacional_final AS
SELECT
    n.*,
    COALESCE(a.epo_100, 0) AS epo_100,
    COALESCE(b.epo_500, 0) AS epo_500
FROM nacional n
LEFT JOIN agg_100 a ON n.id = a.nacional_id
LEFT JOIN agg_500 b ON n.id = b.nacional_id;
""")

# Mostrar una muestra de algunos registros de 'nacional_final' sin cargar todos los datos en Pandas
sample_records = conn.execute("SELECT * FROM nacional_final LIMIT 10").fetchall()
print("Muestra de registros de 'nacional_final':")
for rec in sample_records:
    print(rec)

conn.close()

In [ ]:
# Conectar a la base de datos
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')

# Ejecutar la consulta para obtener 10 registros de la tabla 'nacional'
df_nacional = conn.execute("SELECT * FROM nacional_final LIMIT 10").df()

# Mostrar el DataFrame
print("Muestra de registros de la tabla 'nacional':")
print(df_nacional)

conn.close()

In [ ]:
# Conectar a la base de datos
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')
conn.execute("INSTALL spatial; LOAD spatial;")

# --- Paso A: Agregar personal ocupado por actividad (2 caracteres) en buffers de 100 m ---
conn.execute("""
DROP TABLE IF EXISTS agg_act_100;
CREATE TABLE agg_act_100 AS
SELECT
    n.id AS nacional_id,
    d.codigo_act_2c,
    SUM(d.est_per_ocu) AS sum_est_per_ocu
FROM nacional_buffer_100 n
JOIN denue_est_per_ocu d ON ST_Intersects(n.buffer_100, d.geometry)
GROUP BY n.id, d.codigo_act_2c;
""")

# --- Paso B: Agregar personal ocupado por actividad (2 caracteres) en buffers de 500 m ---
conn.execute("""
DROP TABLE IF EXISTS agg_act_500;
CREATE TABLE agg_act_500 AS
SELECT
    n.id AS nacional_id,
    d.codigo_act_2c,
    SUM(d.est_per_ocu) AS sum_est_per_ocu
FROM nacional_buffer_500 n
JOIN denue_est_per_ocu d ON ST_Intersects(n.buffer_500, d.geometry)
GROUP BY  n.id, d.codigo_act_2c;
""")

# --- Paso C: Crear tabla final para buffers de 100 m pivotando la agregación por actividad ---
conn.execute("""
DROP TABLE IF EXISTS nacional_final_act_100;
CREATE TABLE nacional_final_act_100 AS
SELECT
    n.id, n.klass, n.clee, n.nom_estab, n.cve_ent, n.personal, n.geometry,
    MAX(CASE WHEN a.codigo_act_2c = '51' THEN a.sum_est_per_ocu ELSE 0 END) AS act_51_100,
    MAX(CASE WHEN a.codigo_act_2c = '54' THEN a.sum_est_per_ocu ELSE 0 END) AS act_54_100,
    MAX(CASE WHEN a.codigo_act_2c = '11' THEN a.sum_est_per_ocu ELSE 0 END) AS act_11_100,
    MAX(CASE WHEN a.codigo_act_2c = '22' THEN a.sum_est_per_ocu ELSE 0 END) AS act_22_100,
    MAX(CASE WHEN a.codigo_act_2c = '52' THEN a.sum_est_per_ocu ELSE 0 END) AS act_52_100,
    MAX(CASE WHEN a.codigo_act_2c = '71' THEN a.sum_est_per_ocu ELSE 0 END) AS act_71_100,
    MAX(CASE WHEN a.codigo_act_2c = '43' THEN a.sum_est_per_ocu ELSE 0 END) AS act_43_100,
    MAX(CASE WHEN a.codigo_act_2c = '31' THEN a.sum_est_per_ocu ELSE 0 END) AS act_31_100,
    MAX(CASE WHEN a.codigo_act_2c = '61' THEN a.sum_est_per_ocu ELSE 0 END) AS act_61_100,
    MAX(CASE WHEN a.codigo_act_2c = '46' THEN a.sum_est_per_ocu ELSE 0 END) AS act_46_100,
    MAX(CASE WHEN a.codigo_act_2c = '23' THEN a.sum_est_per_ocu ELSE 0 END) AS act_23_100,
    MAX(CASE WHEN a.codigo_act_2c = '55' THEN a.sum_est_per_ocu ELSE 0 END) AS act_55_100,
    MAX(CASE WHEN a.codigo_act_2c = '93' THEN a.sum_est_per_ocu ELSE 0 END) AS act_93_100,
    MAX(CASE WHEN a.codigo_act_2c = '53' THEN a.sum_est_per_ocu ELSE 0 END) AS act_53_100,
    MAX(CASE WHEN a.codigo_act_2c = '81' THEN a.sum_est_per_ocu ELSE 0 END) AS act_81_100,
    MAX(CASE WHEN a.codigo_act_2c = '33' THEN a.sum_est_per_ocu ELSE 0 END) AS act_33_100,
    MAX(CASE WHEN a.codigo_act_2c = '48' THEN a.sum_est_per_ocu ELSE 0 END) AS act_48_100,
    MAX(CASE WHEN a.codigo_act_2c = '32' THEN a.sum_est_per_ocu ELSE 0 END) AS act_32_100,
    MAX(CASE WHEN a.codigo_act_2c = '56' THEN a.sum_est_per_ocu ELSE 0 END) AS act_56_100,
    MAX(CASE WHEN a.codigo_act_2c = '49' THEN a.sum_est_per_ocu ELSE 0 END) AS act_49_100,
    MAX(CASE WHEN a.codigo_act_2c = '62' THEN a.sum_est_per_ocu ELSE 0 END) AS act_62_100,
    MAX(CASE WHEN a.codigo_act_2c = '21' THEN a.sum_est_per_ocu ELSE 0 END) AS act_21_100,
    MAX(CASE WHEN a.codigo_act_2c = '72' THEN a.sum_est_per_ocu ELSE 0 END) AS act_72_100
FROM nacional n
LEFT JOIN agg_act_100 a ON n.id = a.nacional_id
GROUP BY n.id, n.klass, n.clee, n.nom_estab, n.cve_ent, n.personal, n.geometry;
""")

# --- Paso D: Crear tabla final para buffers de 500 m pivotando la agregación por actividad ---
conn.execute("""
DROP TABLE IF EXISTS nacional_final_act_500;
CREATE TABLE nacional_final_act_500 AS
SELECT
    n.id, n.klass, n.clee, n.nom_estab, n.cve_ent, n.personal, n.geometry,
    MAX(CASE WHEN a.codigo_act_2c = '51' THEN a.sum_est_per_ocu ELSE 0 END) AS act_51_500,
    MAX(CASE WHEN a.codigo_act_2c = '54' THEN a.sum_est_per_ocu ELSE 0 END) AS act_54_500,
    MAX(CASE WHEN a.codigo_act_2c = '11' THEN a.sum_est_per_ocu ELSE 0 END) AS act_11_500,
    MAX(CASE WHEN a.codigo_act_2c = '22' THEN a.sum_est_per_ocu ELSE 0 END) AS act_22_500,
    MAX(CASE WHEN a.codigo_act_2c = '52' THEN a.sum_est_per_ocu ELSE 0 END) AS act_52_500,
    MAX(CASE WHEN a.codigo_act_2c = '71' THEN a.sum_est_per_ocu ELSE 0 END) AS act_71_500,
    MAX(CASE WHEN a.codigo_act_2c = '43' THEN a.sum_est_per_ocu ELSE 0 END) AS act_43_500,
    MAX(CASE WHEN a.codigo_act_2c = '31' THEN a.sum_est_per_ocu ELSE 0 END) AS act_31_500,
    MAX(CASE WHEN a.codigo_act_2c = '61' THEN a.sum_est_per_ocu ELSE 0 END) AS act_61_500,
    MAX(CASE WHEN a.codigo_act_2c = '46' THEN a.sum_est_per_ocu ELSE 0 END) AS act_46_500,
    MAX(CASE WHEN a.codigo_act_2c = '23' THEN a.sum_est_per_ocu ELSE 0 END) AS act_23_500,
    MAX(CASE WHEN a.codigo_act_2c = '55' THEN a.sum_est_per_ocu ELSE 0 END) AS act_55_500,
    MAX(CASE WHEN a.codigo_act_2c = '93' THEN a.sum_est_per_ocu ELSE 0 END) AS act_93_500,
    MAX(CASE WHEN a.codigo_act_2c = '53' THEN a.sum_est_per_ocu ELSE 0 END) AS act_53_500,
    MAX(CASE WHEN a.codigo_act_2c = '81' THEN a.sum_est_per_ocu ELSE 0 END) AS act_81_500,
    MAX(CASE WHEN a.codigo_act_2c = '33' THEN a.sum_est_per_ocu ELSE 0 END) AS act_33_500,
    MAX(CASE WHEN a.codigo_act_2c = '48' THEN a.sum_est_per_ocu ELSE 0 END) AS act_48_500,
    MAX(CASE WHEN a.codigo_act_2c = '32' THEN a.sum_est_per_ocu ELSE 0 END) AS act_32_500,
    MAX(CASE WHEN a.codigo_act_2c = '56' THEN a.sum_est_per_ocu ELSE 0 END) AS act_56_500,
    MAX(CASE WHEN a.codigo_act_2c = '49' THEN a.sum_est_per_ocu ELSE 0 END) AS act_49_500,
    MAX(CASE WHEN a.codigo_act_2c = '62' THEN a.sum_est_per_ocu ELSE 0 END) AS act_62_500,
    MAX(CASE WHEN a.codigo_act_2c = '21' THEN a.sum_est_per_ocu ELSE 0 END) AS act_21_500,
    MAX(CASE WHEN a.codigo_act_2c = '72' THEN a.sum_est_per_ocu ELSE 0 END) AS act_72_500
FROM nacional n
LEFT JOIN agg_act_500 a ON n.id = a.nacional_id
GROUP BY n.id, n.klass, n.clee, n.nom_estab, n.cve_ent, n.personal, n.geometry;
""")

# Mostrar una muestra de registros de la tabla final para 100 m
sample_100 = conn.execute("SELECT * FROM nacional_final_act_100 LIMIT 10").fetchall()
print("Muestra de registros de 'nacional_final_act_100':")
for rec in sample_100:
    print(rec)

# Mostrar una muestra de registros de la tabla final para 500 m
sample_500 = conn.execute("SELECT * FROM nacional_final_act_500 LIMIT 10").fetchall()
print("\nMuestra de registros de 'nacional_final_act_500':")
for rec in sample_500:
    print(rec)

conn.close()


In [ ]:
# Conectar a la base de datos
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')

# Ejecutar la consulta para obtener 10 registros de la tabla 'nacional'
df_nacional = conn.execute("SELECT * FROM nacional_final_act_500 LIMIT 10").df()

# Mostrar el DataFrame
print("Muestra de registros de la tabla 'nacional':")
print(df_nacional)

conn.close()

In [ ]:
# Conectar a la base de datos
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')
conn.execute("INSTALL spatial; LOAD spatial;")

# Crear la tabla final completa uniendo nacional_final con las tablas pivotadas para 100 m y 500 m
conn.execute("""
DROP TABLE IF EXISTS nacional_final_completa;
CREATE TABLE nacional_final_completa AS
SELECT
    n.*,
    -- Variables de buffers de 100 m
    a.act_51_100, a.act_54_100, a.act_11_100, a.act_22_100, a.act_52_100, a.act_71_100,
    a.act_43_100, a.act_31_100, a.act_61_100, a.act_46_100, a.act_23_100, a.act_55_100,
    a.act_93_100, a.act_53_100, a.act_81_100, a.act_33_100, a.act_48_100, a.act_32_100,
    a.act_56_100, a.act_49_100, a.act_62_100, a.act_21_100, a.act_72_100,
    -- Variables de buffers de 500 m
    b.act_51_500, b.act_54_500, b.act_11_500, b.act_22_500, b.act_52_500, b.act_71_500,
    b.act_43_500, b.act_31_500, b.act_61_500, b.act_46_500, b.act_23_500, b.act_55_500,
    b.act_93_500, b.act_53_500, b.act_81_500, b.act_33_500, b.act_48_500, b.act_32_500,
    b.act_56_500, b.act_49_500, b.act_62_500, b.act_21_500, b.act_72_500
FROM nacional_final n
LEFT JOIN nacional_final_act_100 a ON n.id = a.id
LEFT JOIN nacional_final_act_500 b ON n.id = b.id;
""")

# Mostrar una muestra de registros de la tabla completa en un DataFrame
df_completa = conn.execute("SELECT * FROM nacional_final_completa LIMIT 10").df()
print("Muestra de registros de 'nacional_final_completa':")
print(df_completa)

conn.close()


In [ ]:
# Conectar a la base de datos
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')

# Cargar la tabla 'nacional_final_completa' en un DataFrame
df_nacional_final_completa = conn.execute("SELECT * FROM nacional_final_completa").df()

# Mostrar los primeros registros
print("Muestra de registros de 'nacional_final_completa':")
print(df_nacional_final_completa.head())

conn.close()


In [ ]:
df_nacional_final_completa

In [ ]:
# Conectar a la base de datos y extraer los datos con la geometría en WKB
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')
conn.execute("INSTALL spatial; LOAD spatial;")

df = conn.execute("""
    SELECT
        *,
        ST_AsWKB(geometry) AS geom_wkb
    FROM nacional_final_completa
""").df()
conn.close()

# Función segura para cargar la geometría a partir de WKB
def safe_wkb_load(x):
    # Verificar si x es del tipo adecuado (bytes o bytearray)
    if isinstance(x, (bytes, bytearray)):
        try:
            return wkb.loads(x)
        except Exception as e:
            return None
    else:
        return None

# Aplicar la función a la columna 'geom_wkb'
df['geom'] = df['geom_wkb'].apply(safe_wkb_load)

# Eliminar las columnas originales 'geometry' y 'geom_wkb'
df = df.drop(columns=['geometry', 'geom_wkb'])

# Crear el GeoDataFrame usando la columna 'geom' como geometría activa
gdf = gpd.GeoDataFrame(df, geometry='geom', crs="EPSG:4326")

# Exportar a GeoPackage
gdf.to_file("nacional_final_completa.gpkg", driver="GPKG")

print("GeoPackage 'nacional_final_completa.gpkg' creado exitosamente.")


In [ ]:
gdf.to_parquet("nacional_final_completa.geoparquet")

In [ ]:
gdf.to_file("nacional_final_completa.shp")

In [ ]:
# Conectar a la base de datos
db_path = "/content/datos_censo_nacional_s9.duckdb"
conn = duckdb.connect(db_path)

# Obtener listado de tablas
tables = conn.execute("SHOW TABLES").fetchall()

print("Tablas y sus campos:")

# Recorrer cada tabla y obtener sus columnas usando PRAGMA table_info
for table in tables:
    table_name = table[0]
    print(f"\nTabla: {table_name}")
    print("-" * (len(table_name) + 8))
    columns = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
    for col in columns:
        print(f"- {col[1]} ({col[2]})")

conn.close()


In [ ]:
print("")

In [ ]:
# Conectar a la base de datos destino (donde se crearán las tablas de cálculo)
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')
conn.execute("INSTALL spatial; LOAD spatial;")
# --- Paso 1: Crear la tabla de centroides a partir de censo_geo_int ---
# Se adjunta la base de datos fuente (datos_censo_nacional_s9.duckdb)
# Verificar si 'src_db' ya está adjunto usando SHOW DATABASES
attached = conn.execute("SHOW DATABASES;").fetchall()
if not any(row[0] == 'src_db' for row in attached):
    conn.execute("ATTACH DATABASE '/content/datos_censo_nacional_s9.duckdb' AS src_db;")
else:
    print("'src_db' ya está adjunto.")

conn.execute("""
DROP TABLE IF EXISTS censo_geo_int_centroid;
CREATE TABLE censo_geo_int_centroid AS
SELECT
    COALESCE(POBTOT, 0) AS POBTOT,
    COALESCE(POBFEM, 0) AS POBFEM,
    COALESCE(POBMAS, 0) AS POBMAS,

    COALESCE(P_0A2, 0) AS P_0A2,
    COALESCE(P_0A2_F, 0) AS P_0A2_F,
    COALESCE(P_0A2_M, 0) AS P_0A2_M,
    COALESCE(P_3YMAS, 0) AS P_3YMAS,
    COALESCE(P_3YMAS_F, 0) AS P_3YMAS_F,
    COALESCE(P_3YMAS_M, 0) AS P_3YMAS_M,
    COALESCE(P_5YMAS, 0) AS P_5YMAS,
    COALESCE(P_5YMAS_F, 0) AS P_5YMAS_F,
    COALESCE(P_5YMAS_M, 0) AS P_5YMAS_M,
    COALESCE(P_12YMAS, 0) AS P_12YMAS,
    COALESCE(P_12YMAS_F, 0) AS P_12YMAS_F,
    COALESCE(P_12YMAS_M, 0) AS P_12YMAS_M,
    COALESCE(P_15YMAS, 0) AS P_15YMAS,
    COALESCE(P_15YMAS_F, 0) AS P_15YMAS_F,
    COALESCE(P_15YMAS_M, 0) AS P_15YMAS_M,
    COALESCE(P_18YMAS, 0) AS P_18YMAS,
    COALESCE(P_18YMAS_F, 0) AS P_18YMAS_F,
    COALESCE(P_18YMAS_M, 0) AS P_18YMAS_M,
    COALESCE(P_3A5, 0) AS P_3A5,
    COALESCE(P_3A5_F, 0) AS P_3A5_F,
    COALESCE(P_3A5_M, 0) AS P_3A5_M,
    COALESCE(P_6A11, 0) AS P_6A11,
    COALESCE(P_6A11_F, 0) AS P_6A11_F,
    COALESCE(P_6A11_M, 0) AS P_6A11_M,
    COALESCE(P_8A14, 0) AS P_8A14,
    COALESCE(P_8A14_F, 0) AS P_8A14_F,
    COALESCE(P_8A14_M, 0) AS P_8A14_M,
    COALESCE(P_12A14, 0) AS P_12A14,
    COALESCE(P_12A14_F, 0) AS P_12A14_F,
    COALESCE(P_12A14_M, 0) AS P_12A14_M,
    COALESCE(P_15A17, 0) AS P_15A17,
    COALESCE(P_15A17_F, 0) AS P_15A17_F,
    COALESCE(P_15A17_M, 0) AS P_15A17_M,
    COALESCE(P_18A24, 0) AS P_18A24,
    COALESCE(P_18A24_F, 0) AS P_18A24_F,
    COALESCE(P_18A24_M, 0) AS P_18A24_M,
    COALESCE(P_15A49_F, 0) AS P_15A49_F,
    COALESCE(P_60YMAS, 0) AS P_60YMAS,
    COALESCE(P_60YMAS_F, 0) AS P_60YMAS_F,
    COALESCE(P_60YMAS_M, 0) AS P_60YMAS_M,
    COALESCE(REL_H_M, 0) AS REL_H_M,
    COALESCE(POB0_14, 0) AS POB0_14,
    COALESCE(POB15_64, 0) AS POB15_64,
    COALESCE(POB65_MAS, 0) AS POB65_MAS,
    COALESCE(PROM_HNV, 0) AS PROM_HNV,
    COALESCE(PNACENT, 0) AS PNACENT,
    COALESCE(PNACENT_F, 0) AS PNACENT_F,
    COALESCE(PNACENT_M, 0) AS PNACENT_M,
    COALESCE(PNACOE, 0) AS PNACOE,
    COALESCE(PNACOE_F, 0) AS PNACOE_F,
    COALESCE(PNACOE_M, 0) AS PNACOE_M,
    COALESCE(PRES2015, 0) AS PRES2015,
    COALESCE(PRES2015_F, 0) AS PRES2015_F,
    COALESCE(PRES2015_M, 0) AS PRES2015_M,
    COALESCE(PRESOE15, 0) AS PRESOE15,
    COALESCE(PRESOE15_F, 0) AS PRESOE15_F,
    COALESCE(PRESOE15_M, 0) AS PRESOE15_M,
    COALESCE(P3YM_HLI, 0) AS P3YM_HLI,
    COALESCE(P3YM_HLI_F, 0) AS P3YM_HLI_F,
    COALESCE(P3YM_HLI_M, 0) AS P3YM_HLI_M,
    COALESCE(P3HLINHE, 0) AS P3HLINHE,
    COALESCE(P3HLINHE_F, 0) AS P3HLINHE_F,
    COALESCE(P3HLINHE_M, 0) AS P3HLINHE_M,
    COALESCE(P3HLI_HE, 0) AS P3HLI_HE,
    COALESCE(P3HLI_HE_F, 0) AS P3HLI_HE_F,
    COALESCE(P3HLI_HE_M, 0) AS P3HLI_HE_M,
    COALESCE(P5_HLI, 0) AS P5_HLI,
    COALESCE(P5_HLI_NHE, 0) AS P5_HLI_NHE,
    COALESCE(P5_HLI_HE, 0) AS P5_HLI_HE,
    COALESCE(PHOG_IND, 0) AS PHOG_IND,
    COALESCE(POB_AFRO, 0) AS POB_AFRO,
    COALESCE(POB_AFRO_F, 0) AS POB_AFRO_F,
    COALESCE(POB_AFRO_M, 0) AS POB_AFRO_M,
    COALESCE(PCON_DISC, 0) AS PCON_DISC,
    COALESCE(PCDISC_MOT, 0) AS PCDISC_MOT,
    COALESCE(PCDISC_VIS, 0) AS PCDISC_VIS,
    COALESCE(PCDISC_LENG, 0) AS PCDISC_LENG,
    COALESCE(PCDISC_AUD, 0) AS PCDISC_AUD,
    COALESCE(PCDISC_MOT2, 0) AS PCDISC_MOT2,
    COALESCE(PCDISC_MEN, 0) AS PCDISC_MEN,
    COALESCE(PCON_LIMI, 0) AS PCON_LIMI,
    COALESCE(PCLIM_CSB, 0) AS PCLIM_CSB,
    COALESCE(PCLIM_VIS, 0) AS PCLIM_VIS,
    COALESCE(PCLIM_HACO, 0) AS PCLIM_HACO,
    COALESCE(PCLIM_OAUD, 0) AS PCLIM_OAUD,
    COALESCE(PCLIM_MOT2, 0) AS PCLIM_MOT2,
    COALESCE(PCLIM_RE_CO, 0) AS PCLIM_RE_CO,
    COALESCE(PCLIM_PMEN, 0) AS PCLIM_PMEN,
    COALESCE(PSIND_LIM, 0) AS PSIND_LIM,
    COALESCE(P3A5_NOA, 0) AS P3A5_NOA,
    COALESCE(P3A5_NOA_F, 0) AS P3A5_NOA_F,
    COALESCE(P3A5_NOA_M, 0) AS P3A5_NOA_M,
    COALESCE(P6A11_NOA, 0) AS P6A11_NOA,
    COALESCE(P6A11_NOAF, 0) AS P6A11_NOAF,
    COALESCE(P6A11_NOAM, 0) AS P6A11_NOAM,
    COALESCE(P12A14NOA, 0) AS P12A14NOA,
    COALESCE(P12A14NOAF, 0) AS P12A14NOAF,
    COALESCE(P12A14NOAM, 0) AS P12A14NOAM,
    COALESCE(P15A17A, 0) AS P15A17A,
    COALESCE(P15A17A_F, 0) AS P15A17A_F,
    COALESCE(P15A17A_M, 0) AS P15A17A_M,
    COALESCE(P18A24A, 0) AS P18A24A,
    COALESCE(P18A24A_F, 0) AS P18A24A_F,
    COALESCE(P18A24A_M, 0) AS P18A24A_M,
    COALESCE(P8A14AN, 0) AS P8A14AN,
    COALESCE(P8A14AN_F, 0) AS P8A14AN_F,
    COALESCE(P8A14AN_M, 0) AS P8A14AN_M,
    COALESCE(P15YM_AN, 0) AS P15YM_AN,
    COALESCE(P15YM_AN_F, 0) AS P15YM_AN_F,
    COALESCE(P15YM_AN_M, 0) AS P15YM_AN_M,
    COALESCE(P15YM_SE, 0) AS P15YM_SE,
    COALESCE(P15YM_SE_F, 0) AS P15YM_SE_F,
    COALESCE(P15YM_SE_M, 0) AS P15YM_SE_M,
    COALESCE(P15PRI_IN, 0) AS P15PRI_IN,
    COALESCE(P15PRI_INF, 0) AS P15PRI_INF,
    COALESCE(P15PRI_INM, 0) AS P15PRI_INM,
    COALESCE(P15PRI_CO, 0) AS P15PRI_CO,
    COALESCE(P15PRI_COF, 0) AS P15PRI_COF,
    COALESCE(P15PRI_COM, 0) AS P15PRI_COM,
    COALESCE(P15SEC_IN, 0) AS P15SEC_IN,
    COALESCE(P15SEC_INF, 0) AS P15SEC_INF,
    COALESCE(P15SEC_INM, 0) AS P15SEC_INM,
    COALESCE(P15SEC_CO, 0) AS P15SEC_CO,
    COALESCE(P15SEC_COF, 0) AS P15SEC_COF,
    COALESCE(P15SEC_COM, 0) AS P15SEC_COM,
    COALESCE(P18YM_PB, 0) AS P18YM_PB,
    COALESCE(P18YM_PB_F, 0) AS P18YM_PB_F,
    COALESCE(P18YM_PB_M, 0) AS P18YM_PB_M,
    COALESCE(GRAPROES, 0) AS GRAPROES,
    COALESCE(GRAPROES_F, 0) AS GRAPROES_F,
    COALESCE(GRAPROES_M, 0) AS GRAPROES_M,
    COALESCE(PEA, 0) AS PEA,
    COALESCE(PEA_F, 0) AS PEA_F,
    COALESCE(PEA_M, 0) AS PEA_M,
    COALESCE(PE_INAC, 0) AS PE_INAC,
    COALESCE(PE_INAC_F, 0) AS PE_INAC_F,
    COALESCE(PE_INAC_M, 0) AS PE_INAC_M,
    COALESCE(POCUPADA, 0) AS POCUPADA,
    COALESCE(POCUPADA_F, 0) AS POCUPADA_F,
    COALESCE(POCUPADA_M, 0) AS POCUPADA_M,
    COALESCE(PDESOCUP, 0) AS PDESOCUP,
    COALESCE(PDESOCUP_F, 0) AS PDESOCUP_F,
    COALESCE(PDESOCUP_M, 0) AS PDESOCUP_M,
    COALESCE(PSINDER, 0) AS PSINDER,
    COALESCE(PDER_SS, 0) AS PDER_SS,
    COALESCE(PDER_IMSS, 0) AS PDER_IMSS,
    COALESCE(PDER_ISTE, 0) AS PDER_ISTE,
    COALESCE(PDER_ISTEE, 0) AS PDER_ISTEE,
    COALESCE(PAFIL_PDOM, 0) AS PAFIL_PDOM,
    COALESCE(PDER_SEGP, 0) AS PDER_SEGP,
    COALESCE(PDER_IMSSB, 0) AS PDER_IMSSB,
    COALESCE(PAFIL_IPRIV, 0) AS PAFIL_IPRIV,
    COALESCE(PAFIL_OTRAI, 0) AS PAFIL_OTRAI,
    COALESCE(P12YM_SOLT, 0) AS P12YM_SOLT,
    COALESCE(P12YM_CASA, 0) AS P12YM_CASA,
    COALESCE(P12YM_SEPA, 0) AS P12YM_SEPA,
    COALESCE(PCATOLICA, 0) AS PCATOLICA,
    COALESCE(PRO_CRIEVA, 0) AS PRO_CRIEVA,
    COALESCE(POTRAS_REL, 0) AS POTRAS_REL,
    COALESCE(PSIN_RELIG, 0) AS PSIN_RELIG,
    COALESCE(TOTHOG, 0) AS TOTHOG,
    COALESCE(HOGJEF_F, 0) AS HOGJEF_F,
    COALESCE(HOGJEF_M, 0) AS HOGJEF_M,
    COALESCE(POBHOG, 0) AS POBHOG,
    COALESCE(PHOGJEF_F, 0) AS PHOGJEF_F,
    COALESCE(PHOGJEF_M, 0) AS PHOGJEF_M,
    COALESCE(VIVTOT, 0) AS VIVTOT,
    COALESCE(TVIVHAB, 0) AS TVIVHAB,
    COALESCE(TVIVPAR, 0) AS TVIVPAR,
    COALESCE(VIVPAR_HAB, 0) AS VIVPAR_HAB,
    COALESCE(VIVPARH_CV, 0) AS VIVPARH_CV,
    COALESCE(TVIVPARHAB, 0) AS TVIVPARHAB,
    COALESCE(VIVPAR_DES, 0) AS VIVPAR_DES,
    COALESCE(VIVPAR_UT, 0) AS VIVPAR_UT,
    COALESCE(OCUPVIVPAR, 0) AS OCUPVIVPAR,
    COALESCE(PROM_OCUP, 0) AS PROM_OCUP,
    COALESCE(PRO_OCUP_C, 0) AS PRO_OCUP_C,
    COALESCE(VPH_PISODT, 0) AS VPH_PISODT,
    COALESCE(VPH_PISOTI, 0) AS VPH_PISOTI,
    COALESCE(VPH_1DOR, 0) AS VPH_1DOR,
    COALESCE(VPH_2YMASD, 0) AS VPH_2YMASD,
    COALESCE(VPH_1CUART, 0) AS VPH_1CUART,
    COALESCE(VPH_2CUART, 0) AS VPH_2CUART,
    COALESCE(VPH_3YMASC, 0) AS VPH_3YMASC,
    COALESCE(VPH_C_ELEC, 0) AS VPH_C_ELEC,
    COALESCE(VPH_S_ELEC, 0) AS VPH_S_ELEC,
    COALESCE(VPH_AGUADV, 0) AS VPH_AGUADV,
    COALESCE(VPH_AEASP, 0) AS VPH_AEASP,
    COALESCE(VPH_AGUAFV, 0) AS VPH_AGUAFV,
    COALESCE(VPH_TINACO, 0) AS VPH_TINACO,
    COALESCE(VPH_CISTER, 0) AS VPH_CISTER,
    COALESCE(VPH_EXCSA, 0) AS VPH_EXCSA,
    COALESCE(VPH_LETR, 0) AS VPH_LETR,
    COALESCE(VPH_DRENAJ, 0) AS VPH_DRENAJ,
    COALESCE(VPH_NODREN, 0) AS VPH_NODREN,
    COALESCE(VPH_C_SERV, 0) AS VPH_C_SERV,
    COALESCE(VPH_NDEAED, 0) AS VPH_NDEAED,
    COALESCE(VPH_DSADMA, 0) AS VPH_DSADMA,
    COALESCE(VPH_NDACMM, 0) AS VPH_NDACMM,
    COALESCE(VPH_SNBIEN, 0) AS VPH_SNBIEN,
    COALESCE(VPH_REFRI, 0) AS VPH_REFRI,
    COALESCE(VPH_LAVAD, 0) AS VPH_LAVAD,
    COALESCE(VPH_HMICRO, 0) AS VPH_HMICRO,
    COALESCE(VPH_AUTOM, 0) AS VPH_AUTOM,
    COALESCE(VPH_MOTO, 0) AS VPH_MOTO,
    COALESCE(VPH_BICI, 0) AS VPH_BICI,
    COALESCE(VPH_RADIO, 0) AS VPH_RADIO,
    COALESCE(VPH_TV, 0) AS VPH_TV,
    COALESCE(VPH_PC, 0) AS VPH_PC,
    COALESCE(VPH_TELEF, 0) AS VPH_TELEF,
    COALESCE(VPH_CEL, 0) AS VPH_CEL,
    COALESCE(VPH_INTER, 0) AS VPH_INTER,
    COALESCE(VPH_STVP, 0) AS VPH_STVP,
    COALESCE(VPH_SPMVPI, 0) AS VPH_SPMVPI,
    COALESCE(VPH_CVJ, 0) AS VPH_CVJ,
    COALESCE(VPH_SINRTV, 0) AS VPH_SINRTV,
    COALESCE(VPH_SINLTC, 0) AS VPH_SINLTC,
    COALESCE(VPH_SINCINT, 0) AS VPH_SINCINT,
    COALESCE(VPH_SINTIC, 0) AS VPH_SINTIC,
    ST_Centroid(geometry) AS geometry
FROM src_db.censo_geo_int;
""")
print("Tabla 'censo_geo_int_centroid' creada.")

# --- Paso 2: Crear las tablas de buffers a partir de los centroides ---
conn.execute("""
DROP TABLE IF EXISTS censo_geo_int_centroid_buffer_100;
CREATE TABLE censo_geo_int_centroid_buffer_100 AS
SELECT *,
       ST_Buffer(geometry, 0.000898) AS buffer_100
FROM censo_geo_int_centroid;
""")
conn.execute("""
DROP TABLE IF EXISTS censo_geo_int_centroid_buffer_500;
CREATE TABLE censo_geo_int_centroid_buffer_500 AS
SELECT *,
       ST_Buffer(geometry, 0.00449) AS buffer_500
FROM censo_geo_int_centroid;
""")
print("Tablas de buffers (100 m y 500 m) creadas a partir de los centroides.")

# --- Paso 3: Agregar (sumar) los campos numéricos de censo para cada punto de nacional_final_completa ---
# Usamos ST_Intersects para relacionar la geometría de cada punto con el buffer de censo.
conn.execute("""
DROP TABLE IF EXISTS censo_agg_100;
CREATE TABLE censo_agg_100 AS
SELECT
    n.id AS national_id,
    SUM(c.POBTOT) AS sum_POBTOT,
    SUM(c.POBFEM) AS sum_POBFEM,
    SUM(c.POBMAS) AS sum_POBMAS,
    SUM(c.P_0A2) AS sum_P_0A2,
    SUM(c.P_0A2_F) AS sum_P_0A2_F,
    SUM(c.P_0A2_M) AS sum_P_0A2_M,
    SUM(c.P_3YMAS) AS sum_P_3YMAS,
    SUM(c.P_3YMAS_F) AS sum_P_3YMAS_F,
    SUM(c.P_3YMAS_M) AS sum_P_3YMAS_M,
    SUM(c.P_5YMAS) AS sum_P_5YMAS,
    SUM(c.P_5YMAS_F) AS sum_P_5YMAS_F,
    SUM(c.P_5YMAS_M) AS sum_P_5YMAS_M,
    SUM(c.P_12YMAS) AS sum_P_12YMAS,
    SUM(c.P_12YMAS_F) AS sum_P_12YMAS_F,
    SUM(c.P_12YMAS_M) AS sum_P_12YMAS_M,
    SUM(c.P_15YMAS) AS sum_P_15YMAS,
    SUM(c.P_15YMAS_F) AS sum_P_15YMAS_F,
    SUM(c.P_15YMAS_M) AS sum_P_15YMAS_M,
    SUM(c.P_18YMAS) AS sum_P_18YMAS,
    SUM(c.P_18YMAS_F) AS sum_P_18YMAS_F,
    SUM(c.P_18YMAS_M) AS sum_P_18YMAS_M,
    SUM(c.P_3A5) AS sum_P_3A5,
    SUM(c.P_3A5_F) AS sum_P_3A5_F,
    SUM(c.P_3A5_M) AS sum_P_3A5_M,
    SUM(c.P_6A11) AS sum_P_6A11,
    SUM(c.P_6A11_F) AS sum_P_6A11_F,
    SUM(c.P_6A11_M) AS sum_P_6A11_M,
    SUM(c.P_8A14) AS sum_P_8A14,
    SUM(c.P_8A14_F) AS sum_P_8A14_F,
    SUM(c.P_8A14_M) AS sum_P_8A14_M,
    SUM(c.P_12A14) AS sum_P_12A14,
    SUM(c.P_12A14_F) AS sum_P_12A14_F,
    SUM(c.P_12A14_M) AS sum_P_12A14_M,
    SUM(c.P_15A17) AS sum_P_15A17,
    SUM(c.P_15A17_F) AS sum_P_15A17_F,
    SUM(c.P_15A17_M) AS sum_P_15A17_M,
    SUM(c.P_18A24) AS sum_P_18A24,
    SUM(c.P_18A24_F) AS sum_P_18A24_F,
    SUM(c.P_18A24_M) AS sum_P_18A24_M,
    SUM(c.P_15A49_F) AS sum_P_15A49_F,
    SUM(c.P_60YMAS) AS sum_P_60YMAS,
    SUM(c.P_60YMAS_F) AS sum_P_60YMAS_F,
    SUM(c.P_60YMAS_M) AS sum_P_60YMAS_M,
    SUM(c.REL_H_M) AS sum_REL_H_M,
    SUM(c.POB0_14) AS sum_POB0_14,
    SUM(c.POB15_64) AS sum_POB15_64,
    SUM(c.POB65_MAS) AS sum_POB65_MAS,
    SUM(c.PROM_HNV) AS sum_PROM_HNV,
    SUM(c.PNACENT) AS sum_PNACENT,
    SUM(c.PNACENT_F) AS sum_PNACENT_F,
    SUM(c.PNACENT_M) AS sum_PNACENT_M,
    SUM(c.PNACOE) AS sum_PNACOE,
    SUM(c.PNACOE_F) AS sum_PNACOE_F,
    SUM(c.PNACOE_M) AS sum_PNACOE_M,
    SUM(c.PRES2015) AS sum_PRES2015,
    SUM(c.PRES2015_F) AS sum_PRES2015_F,
    SUM(c.PRES2015_M) AS sum_PRES2015_M,
    SUM(c.PRESOE15) AS sum_PRESOE15,
    SUM(c.PRESOE15_F) AS sum_PRESOE15_F,
    SUM(c.PRESOE15_M) AS sum_PRESOE15_M,
    SUM(c.P3YM_HLI) AS sum_P3YM_HLI,
    SUM(c.P3YM_HLI_F) AS sum_P3YM_HLI_F,
    SUM(c.P3YM_HLI_M) AS sum_P3YM_HLI_M,
    SUM(c.P3HLINHE) AS sum_P3HLINHE,
    SUM(c.P3HLINHE_F) AS sum_P3HLINHE_F,
    SUM(c.P3HLINHE_M) AS sum_P3HLINHE_M,
    SUM(c.P3HLI_HE) AS sum_P3HLI_HE,
    SUM(c.P3HLI_HE_F) AS sum_P3HLI_HE_F,
    SUM(c.P3HLI_HE_M) AS sum_P3HLI_HE_M,
    SUM(c.P5_HLI) AS sum_P5_HLI,
    SUM(c.P5_HLI_NHE) AS sum_P5_HLI_NHE,
    SUM(c.P5_HLI_HE) AS sum_P5_HLI_HE,
    SUM(c.PHOG_IND) AS sum_PHOG_IND,
    SUM(c.POB_AFRO) AS sum_POB_AFRO,
    SUM(c.POB_AFRO_F) AS sum_POB_AFRO_F,
    SUM(c.POB_AFRO_M) AS sum_POB_AFRO_M,
    SUM(c.PCON_DISC) AS sum_PCON_DISC,
    SUM(c.PCDISC_MOT) AS sum_PCDISC_MOT,
    SUM(c.PCDISC_VIS) AS sum_PCDISC_VIS,
    SUM(c.PCDISC_LENG) AS sum_PCDISC_LENG,
    SUM(c.PCDISC_AUD) AS sum_PCDISC_AUD,
    SUM(c.PCDISC_MOT2) AS sum_PCDISC_MOT2,
    SUM(c.PCDISC_MEN) AS sum_PCDISC_MEN,
    SUM(c.PCON_LIMI) AS sum_PCON_LIMI,
    SUM(c.PCLIM_CSB) AS sum_PCLIM_CSB,
    SUM(c.PCLIM_VIS) AS sum_PCLIM_VIS,
    SUM(c.PCLIM_HACO) AS sum_PCLIM_HACO,
    SUM(c.PCLIM_OAUD) AS sum_PCLIM_OAUD,
    SUM(c.PCLIM_MOT2) AS sum_PCLIM_MOT2,
    SUM(c.PCLIM_RE_CO) AS sum_PCLIM_RE_CO,
    SUM(c.PCLIM_PMEN) AS sum_PCLIM_PMEN,
    SUM(c.PSIND_LIM) AS sum_PSIND_LIM,
    SUM(c.P3A5_NOA) AS sum_P3A5_NOA,
    SUM(c.P3A5_NOA_F) AS sum_P3A5_NOA_F,
    SUM(c.P3A5_NOA_M) AS sum_P3A5_NOA_M,
    SUM(c.P6A11_NOA) AS sum_P6A11_NOA,
    SUM(c.P6A11_NOAF) AS sum_P6A11_NOAF,
    SUM(c.P6A11_NOAM) AS sum_P6A11_NOAM,
    SUM(c.P12A14NOA) AS sum_P12A14NOA,
    SUM(c.P12A14NOAF) AS sum_P12A14NOAF,
    SUM(c.P12A14NOAM) AS sum_P12A14NOAM,
    SUM(c.P15A17A) AS sum_P15A17A,
    SUM(c.P15A17A_F) AS sum_P15A17A_F,
    SUM(c.P15A17A_M) AS sum_P15A17A_M,
    SUM(c.P18A24A) AS sum_P18A24A,
    SUM(c.P18A24A_F) AS sum_P18A24A_F,
    SUM(c.P18A24A_M) AS sum_P18A24A_M,
    SUM(c.P8A14AN) AS sum_P8A14AN,
    SUM(c.P8A14AN_F) AS sum_P8A14AN_F,
    SUM(c.P8A14AN_M) AS sum_P8A14AN_M,
    SUM(c.P15YM_AN) AS sum_P15YM_AN,
    SUM(c.P15YM_AN_F) AS sum_P15YM_AN_F,
    SUM(c.P15YM_AN_M) AS sum_P15YM_AN_M,
    SUM(c.P15YM_SE) AS sum_P15YM_SE,
    SUM(c.P15YM_SE_F) AS sum_P15YM_SE_F,
    SUM(c.P15YM_SE_M) AS sum_P15YM_SE_M,
    SUM(c.P15PRI_IN) AS sum_P15PRI_IN,
    SUM(c.P15PRI_INF) AS sum_P15PRI_INF,
    SUM(c.P15PRI_INM) AS sum_P15PRI_INM,
    SUM(c.P15PRI_CO) AS sum_P15PRI_CO,
    SUM(c.P15PRI_COF) AS sum_P15PRI_COF,
    SUM(c.P15PRI_COM) AS sum_P15PRI_COM,
    SUM(c.P15SEC_IN) AS sum_P15SEC_IN,
    SUM(c.P15SEC_INF) AS sum_P15SEC_INF,
    SUM(c.P15SEC_INM) AS sum_P15SEC_INM,
    SUM(c.P15SEC_CO) AS sum_P15SEC_CO,
    SUM(c.P15SEC_COF) AS sum_P15SEC_COF,
    SUM(c.P15SEC_COM) AS sum_P15SEC_COM,
    SUM(c.P18YM_PB) AS sum_P18YM_PB,
    SUM(c.P18YM_PB_F) AS sum_P18YM_PB_F,
    SUM(c.P18YM_PB_M) AS sum_P18YM_PB_M,
    SUM(c.GRAPROES) AS sum_GRAPROES,
    SUM(c.GRAPROES_F) AS sum_GRAPROES_F,
    SUM(c.GRAPROES_M) AS sum_GRAPROES_M,
    SUM(c.PEA) AS sum_PEA,
    SUM(c.PEA_F) AS sum_PEA_F,
    SUM(c.PEA_M) AS sum_PEA_M,
    SUM(c.PE_INAC) AS sum_PE_INAC,
    SUM(c.PE_INAC_F) AS sum_PE_INAC_F,
    SUM(c.PE_INAC_M) AS sum_PE_INAC_M,
    SUM(c.POCUPADA) AS sum_POCUPADA,
    SUM(c.POCUPADA_F) AS sum_POCUPADA_F,
    SUM(c.POCUPADA_M) AS sum_POCUPADA_M,
    SUM(c.PDESOCUP) AS sum_PDESOCUP,
    SUM(c.PDESOCUP_F) AS sum_PDESOCUP_F,
    SUM(c.PDESOCUP_M) AS sum_PDESOCUP_M,
    SUM(c.PSINDER) AS sum_PSINDER,
    SUM(c.PDER_SS) AS sum_PDER_SS,
    SUM(c.PDER_IMSS) AS sum_PDER_IMSS,
    SUM(c.PDER_ISTE) AS sum_PDER_ISTE,
    SUM(c.PDER_ISTEE) AS sum_PDER_ISTEE,
    SUM(c.PAFIL_PDOM) AS sum_PAFIL_PDOM,
    SUM(c.PDER_SEGP) AS sum_PDER_SEGP,
    SUM(c.PDER_IMSSB) AS sum_PDER_IMSSB,
    SUM(c.PAFIL_IPRIV) AS sum_PAFIL_IPRIV,
    SUM(c.PAFIL_OTRAI) AS sum_PAFIL_OTRAI,
    SUM(c.P12YM_SOLT) AS sum_P12YM_SOLT,
    SUM(c.P12YM_CASA) AS sum_P12YM_CASA,
    SUM(c.P12YM_SEPA) AS sum_P12YM_SEPA,
    SUM(c.PCATOLICA) AS sum_PCATOLICA,
    SUM(c.PRO_CRIEVA) AS sum_PRO_CRIEVA,
    SUM(c.POTRAS_REL) AS sum_POTRAS_REL,
    SUM(c.PSIN_RELIG) AS sum_PSIN_RELIG,
    SUM(c.TOTHOG) AS sum_TOTHOG,
    SUM(c.HOGJEF_F) AS sum_HOGJEF_F,
    SUM(c.HOGJEF_M) AS sum_HOGJEF_M,
    SUM(c.POBHOG) AS sum_POBHOG,
    SUM(c.PHOGJEF_F) AS sum_PHOGJEF_F,
    SUM(c.PHOGJEF_M) AS sum_PHOGJEF_M,
    SUM(c.VIVTOT) AS sum_VIVTOT,
    SUM(c.TVIVHAB) AS sum_TVIVHAB,
    SUM(c.TVIVPAR) AS sum_TVIVPAR,
    SUM(c.VIVPAR_HAB) AS sum_VIVPAR_HAB,
    SUM(c.VIVPARH_CV) AS sum_VIVPARH_CV,
    SUM(c.TVIVPARHAB) AS sum_TVIVPARHAB,
    SUM(c.VIVPAR_DES) AS sum_VIVPAR_DES,
    SUM(c.VIVPAR_UT) AS sum_VIVPAR_UT,
    SUM(c.OCUPVIVPAR) AS sum_OCUPVIVPAR,
    SUM(c.PROM_OCUP) AS sum_PROM_OCUP,
    SUM(c.PRO_OCUP_C) AS sum_PRO_OCUP_C,
    SUM(c.VPH_PISODT) AS sum_VPH_PISODT,
    SUM(c.VPH_PISOTI) AS sum_VPH_PISOTI,
    SUM(c.VPH_1DOR) AS sum_VPH_1DOR,
    SUM(c.VPH_2YMASD) AS sum_VPH_2YMASD,
    SUM(c.VPH_1CUART) AS sum_VPH_1CUART,
    SUM(c.VPH_2CUART) AS sum_VPH_2CUART,
    SUM(c.VPH_3YMASC) AS sum_VPH_3YMASC,
    SUM(c.VPH_C_ELEC) AS sum_VPH_C_ELEC,
    SUM(c.VPH_S_ELEC) AS sum_VPH_S_ELEC,
    SUM(c.VPH_AGUADV) AS sum_VPH_AGUADV,
    SUM(c.VPH_AEASP) AS sum_VPH_AEASP,
    SUM(c.VPH_AGUAFV) AS sum_VPH_AGUAFV,
    SUM(c.VPH_TINACO) AS sum_VPH_TINACO,
    SUM(c.VPH_CISTER) AS sum_VPH_CISTER,
    SUM(c.VPH_EXCSA) AS sum_VPH_EXCSA,
    SUM(c.VPH_LETR) AS sum_VPH_LETR,
    SUM(c.VPH_DRENAJ) AS sum_VPH_DRENAJ,
    SUM(c.VPH_NODREN) AS sum_VPH_NODREN,
    SUM(c.VPH_C_SERV) AS sum_VPH_C_SERV,
    SUM(c.VPH_NDEAED) AS sum_VPH_NDEAED,
    SUM(c.VPH_DSADMA) AS sum_VPH_DSADMA,
    SUM(c.VPH_NDACMM) AS sum_VPH_NDACMM,
    SUM(c.VPH_SNBIEN) AS sum_VPH_SNBIEN,
    SUM(c.VPH_REFRI) AS sum_VPH_REFRI,
    SUM(c.VPH_LAVAD) AS sum_VPH_LAVAD,
    SUM(c.VPH_HMICRO) AS sum_VPH_HMICRO,
    SUM(c.VPH_AUTOM) AS sum_VPH_AUTOM,
    SUM(c.VPH_MOTO) AS sum_VPH_MOTO,
    SUM(c.VPH_BICI) AS sum_VPH_BICI,
    SUM(c.VPH_RADIO) AS sum_VPH_RADIO,
    SUM(c.VPH_TV) AS sum_VPH_TV,
    SUM(c.VPH_PC) AS sum_VPH_PC,
    SUM(c.VPH_TELEF) AS sum_VPH_TELEF,
    SUM(c.VPH_CEL) AS sum_VPH_CEL,
    SUM(c.VPH_INTER) AS sum_VPH_INTER,
    SUM(c.VPH_STVP) AS sum_VPH_STVP,
    SUM(c.VPH_SPMVPI) AS sum_VPH_SPMVPI,
    SUM(c.VPH_CVJ) AS sum_VPH_CVJ,
    SUM(c.VPH_SINRTV) AS sum_VPH_SINRTV,
    SUM(c.VPH_SINLTC) AS sum_VPH_SINLTC,
    SUM(c.VPH_SINCINT) AS sum_VPH_SINCINT,
    SUM(c.VPH_SINTIC) AS sum_VPH_SINTIC
FROM nacional_final_completa n
JOIN censo_geo_int_centroid_buffer_100 c
  ON ST_Intersects(n.geometry, c.buffer_100)
GROUP BY n.id;
""")

conn.execute("""
DROP TABLE IF EXISTS censo_agg_500;
CREATE TABLE censo_agg_500 AS
SELECT
    n.id AS national_id,
    SUM(c.POBTOT) AS sum_POBTOT,
    SUM(c.POBFEM) AS sum_POBFEM,
    SUM(c.POBMAS) AS sum_POBMAS,
    SUM(c.P_0A2) AS sum_P_0A2,
    SUM(c.P_0A2_F) AS sum_P_0A2_F,
    SUM(c.P_0A2_M) AS sum_P_0A2_M,
    SUM(c.P_3YMAS) AS sum_P_3YMAS,
    SUM(c.P_3YMAS_F) AS sum_P_3YMAS_F,
    SUM(c.P_3YMAS_M) AS sum_P_3YMAS_M,
    SUM(c.P_5YMAS) AS sum_P_5YMAS,
    SUM(c.P_5YMAS_F) AS sum_P_5YMAS_F,
    SUM(c.P_5YMAS_M) AS sum_P_5YMAS_M,
    SUM(c.P_12YMAS) AS sum_P_12YMAS,
    SUM(c.P_12YMAS_F) AS sum_P_12YMAS_F,
    SUM(c.P_12YMAS_M) AS sum_P_12YMAS_M,
    SUM(c.P_15YMAS) AS sum_P_15YMAS,
    SUM(c.P_15YMAS_F) AS sum_P_15YMAS_F,
    SUM(c.P_15YMAS_M) AS sum_P_15YMAS_M,
    SUM(c.P_18YMAS) AS sum_P_18YMAS,
    SUM(c.P_18YMAS_F) AS sum_P_18YMAS_F,
    SUM(c.P_18YMAS_M) AS sum_P_18YMAS_M,
    SUM(c.P_3A5) AS sum_P_3A5,
    SUM(c.P_3A5_F) AS sum_P_3A5_F,
    SUM(c.P_3A5_M) AS sum_P_3A5_M,
    SUM(c.P_6A11) AS sum_P_6A11,
    SUM(c.P_6A11_F) AS sum_P_6A11_F,
    SUM(c.P_6A11_M) AS sum_P_6A11_M,
    SUM(c.P_8A14) AS sum_P_8A14,
    SUM(c.P_8A14_F) AS sum_P_8A14_F,
    SUM(c.P_8A14_M) AS sum_P_8A14_M,
    SUM(c.P_12A14) AS sum_P_12A14,
    SUM(c.P_12A14_F) AS sum_P_12A14_F,
    SUM(c.P_12A14_M) AS sum_P_12A14_M,
    SUM(c.P_15A17) AS sum_P_15A17,
    SUM(c.P_15A17_F) AS sum_P_15A17_F,
    SUM(c.P_15A17_M) AS sum_P_15A17_M,
    SUM(c.P_18A24) AS sum_P_18A24,
    SUM(c.P_18A24_F) AS sum_P_18A24_F,
    SUM(c.P_18A24_M) AS sum_P_18A24_M,
    SUM(c.P_15A49_F) AS sum_P_15A49_F,
    SUM(c.P_60YMAS) AS sum_P_60YMAS,
    SUM(c.P_60YMAS_F) AS sum_P_60YMAS_F,
    SUM(c.P_60YMAS_M) AS sum_P_60YMAS_M,
    SUM(c.REL_H_M) AS sum_REL_H_M,
    SUM(c.POB0_14) AS sum_POB0_14,
    SUM(c.POB15_64) AS sum_POB15_64,
    SUM(c.POB65_MAS) AS sum_POB65_MAS,
    SUM(c.PROM_HNV) AS sum_PROM_HNV,
    SUM(c.PNACENT) AS sum_PNACENT,
    SUM(c.PNACENT_F) AS sum_PNACENT_F,
    SUM(c.PNACENT_M) AS sum_PNACENT_M,
    SUM(c.PNACOE) AS sum_PNACOE,
    SUM(c.PNACOE_F) AS sum_PNACOE_F,
    SUM(c.PNACOE_M) AS sum_PNACOE_M,
    SUM(c.PRES2015) AS sum_PRES2015,
    SUM(c.PRES2015_F) AS sum_PRES2015_F,
    SUM(c.PRES2015_M) AS sum_PRES2015_M,
    SUM(c.PRESOE15) AS sum_PRESOE15,
    SUM(c.PRESOE15_F) AS sum_PRESOE15_F,
    SUM(c.PRESOE15_M) AS sum_PRESOE15_M,
    SUM(c.P3YM_HLI) AS sum_P3YM_HLI,
    SUM(c.P3YM_HLI_F) AS sum_P3YM_HLI_F,
    SUM(c.P3YM_HLI_M) AS sum_P3YM_HLI_M,
    SUM(c.P3HLINHE) AS sum_P3HLINHE,
    SUM(c.P3HLINHE_F) AS sum_P3HLINHE_F,
    SUM(c.P3HLINHE_M) AS sum_P3HLINHE_M,
    SUM(c.P3HLI_HE) AS sum_P3HLI_HE,
    SUM(c.P3HLI_HE_F) AS sum_P3HLI_HE_F,
    SUM(c.P3HLI_HE_M) AS sum_P3HLI_HE_M,
    SUM(c.P5_HLI) AS sum_P5_HLI,
    SUM(c.P5_HLI_NHE) AS sum_P5_HLI_NHE,
    SUM(c.P5_HLI_HE) AS sum_P5_HLI_HE,
    SUM(c.PHOG_IND) AS sum_PHOG_IND,
    SUM(c.POB_AFRO) AS sum_POB_AFRO,
    SUM(c.POB_AFRO_F) AS sum_POB_AFRO_F,
    SUM(c.POB_AFRO_M) AS sum_POB_AFRO_M,
    SUM(c.PCON_DISC) AS sum_PCON_DISC,
    SUM(c.PCDISC_MOT) AS sum_PCDISC_MOT,
    SUM(c.PCDISC_VIS) AS sum_PCDISC_VIS,
    SUM(c.PCDISC_LENG) AS sum_PCDISC_LENG,
    SUM(c.PCDISC_AUD) AS sum_PCDISC_AUD,
    SUM(c.PCDISC_MOT2) AS sum_PCDISC_MOT2,
    SUM(c.PCDISC_MEN) AS sum_PCDISC_MEN,
    SUM(c.PCON_LIMI) AS sum_PCON_LIMI,
    SUM(c.PCLIM_CSB) AS sum_PCLIM_CSB,
    SUM(c.PCLIM_VIS) AS sum_PCLIM_VIS,
    SUM(c.PCLIM_HACO) AS sum_PCLIM_HACO,
    SUM(c.PCLIM_OAUD) AS sum_PCLIM_OAUD,
    SUM(c.PCLIM_MOT2) AS sum_PCLIM_MOT2,
    SUM(c.PCLIM_RE_CO) AS sum_PCLIM_RE_CO,
    SUM(c.PCLIM_PMEN) AS sum_PCLIM_PMEN,
    SUM(c.PSIND_LIM) AS sum_PSIND_LIM,
    SUM(c.P3A5_NOA) AS sum_P3A5_NOA,
    SUM(c.P3A5_NOA_F) AS sum_P3A5_NOA_F,
    SUM(c.P3A5_NOA_M) AS sum_P3A5_NOA_M,
    SUM(c.P6A11_NOA) AS sum_P6A11_NOA,
    SUM(c.P6A11_NOAF) AS sum_P6A11_NOAF,
    SUM(c.P6A11_NOAM) AS sum_P6A11_NOAM,
    SUM(c.P12A14NOA) AS sum_P12A14NOA,
    SUM(c.P12A14NOAF) AS sum_P12A14NOAF,
    SUM(c.P12A14NOAM) AS sum_P12A14NOAM,
    SUM(c.P15A17A) AS sum_P15A17A,
    SUM(c.P15A17A_F) AS sum_P15A17A_F,
    SUM(c.P15A17A_M) AS sum_P15A17A_M,
    SUM(c.P18A24A) AS sum_P18A24A,
    SUM(c.P18A24A_F) AS sum_P18A24A_F,
    SUM(c.P18A24A_M) AS sum_P18A24A_M,
    SUM(c.P8A14AN) AS sum_P8A14AN,
    SUM(c.P8A14AN_F) AS sum_P8A14AN_F,
    SUM(c.P8A14AN_M) AS sum_P8A14AN_M,
    SUM(c.P15YM_AN) AS sum_P15YM_AN,
    SUM(c.P15YM_AN_F) AS sum_P15YM_AN_F,
    SUM(c.P15YM_AN_M) AS sum_P15YM_AN_M,
    SUM(c.P15YM_SE) AS sum_P15YM_SE,
    SUM(c.P15YM_SE_F) AS sum_P15YM_SE_F,
    SUM(c.P15YM_SE_M) AS sum_P15YM_SE_M,
    SUM(c.P15PRI_IN) AS sum_P15PRI_IN,
    SUM(c.P15PRI_INF) AS sum_P15PRI_INF,
    SUM(c.P15PRI_INM) AS sum_P15PRI_INM,
    SUM(c.P15PRI_CO) AS sum_P15PRI_CO,
    SUM(c.P15PRI_COF) AS sum_P15PRI_COF,
    SUM(c.P15PRI_COM) AS sum_P15PRI_COM,
    SUM(c.P15SEC_IN) AS sum_P15SEC_IN,
    SUM(c.P15SEC_INF) AS sum_P15SEC_INF,
    SUM(c.P15SEC_INM) AS sum_P15SEC_INM,
    SUM(c.P15SEC_CO) AS sum_P15SEC_CO,
    SUM(c.P15SEC_COF) AS sum_P15SEC_COF,
    SUM(c.P15SEC_COM) AS sum_P15SEC_COM,
    SUM(c.P18YM_PB) AS sum_P18YM_PB,
    SUM(c.P18YM_PB_F) AS sum_P18YM_PB_F,
    SUM(c.P18YM_PB_M) AS sum_P18YM_PB_M,
    SUM(c.GRAPROES) AS sum_GRAPROES,
    SUM(c.GRAPROES_F) AS sum_GRAPROES_F,
    SUM(c.GRAPROES_M) AS sum_GRAPROES_M,
    SUM(c.PEA) AS sum_PEA,
    SUM(c.PEA_F) AS sum_PEA_F,
    SUM(c.PEA_M) AS sum_PEA_M,
    SUM(c.PE_INAC) AS sum_PE_INAC,
    SUM(c.PE_INAC_F) AS sum_PE_INAC_F,
    SUM(c.PE_INAC_M) AS sum_PE_INAC_M,
    SUM(c.POCUPADA) AS sum_POCUPADA,
    SUM(c.POCUPADA_F) AS sum_POCUPADA_F,
    SUM(c.POCUPADA_M) AS sum_POCUPADA_M,
    SUM(c.PDESOCUP) AS sum_PDESOCUP,
    SUM(c.PDESOCUP_F) AS sum_PDESOCUP_F,
    SUM(c.PDESOCUP_M) AS sum_PDESOCUP_M,
    SUM(c.PSINDER) AS sum_PSINDER,
    SUM(c.PDER_SS) AS sum_PDER_SS,
    SUM(c.PDER_IMSS) AS sum_PDER_IMSS,
    SUM(c.PDER_ISTE) AS sum_PDER_ISTE,
    SUM(c.PDER_ISTEE) AS sum_PDER_ISTEE,
    SUM(c.PAFIL_PDOM) AS sum_PAFIL_PDOM,
    SUM(c.PDER_SEGP) AS sum_PDER_SEGP,
    SUM(c.PDER_IMSSB) AS sum_PDER_IMSSB,
    SUM(c.PAFIL_IPRIV) AS sum_PAFIL_IPRIV,
    SUM(c.PAFIL_OTRAI) AS sum_PAFIL_OTRAI,
    SUM(c.P12YM_SOLT) AS sum_P12YM_SOLT,
    SUM(c.P12YM_CASA) AS sum_P12YM_CASA,
    SUM(c.P12YM_SEPA) AS sum_P12YM_SEPA,
    SUM(c.PCATOLICA) AS sum_PCATOLICA,
    SUM(c.PRO_CRIEVA) AS sum_PRO_CRIEVA,
    SUM(c.POTRAS_REL) AS sum_POTRAS_REL,
    SUM(c.PSIN_RELIG) AS sum_PSIN_RELIG,
    SUM(c.TOTHOG) AS sum_TOTHOG,
    SUM(c.HOGJEF_F) AS sum_HOGJEF_F,
    SUM(c.HOGJEF_M) AS sum_HOGJEF_M,
    SUM(c.POBHOG) AS sum_POBHOG,
    SUM(c.PHOGJEF_F) AS sum_PHOGJEF_F,
    SUM(c.PHOGJEF_M) AS sum_PHOGJEF_M,
    SUM(c.VIVTOT) AS sum_VIVTOT,
    SUM(c.TVIVHAB) AS sum_TVIVHAB,
    SUM(c.TVIVPAR) AS sum_TVIVPAR,
    SUM(c.VIVPAR_HAB) AS sum_VIVPAR_HAB,
    SUM(c.VIVPARH_CV) AS sum_VIVPARH_CV,
    SUM(c.TVIVPARHAB) AS sum_TVIVPARHAB,
    SUM(c.VIVPAR_DES) AS sum_VIVPAR_DES,
    SUM(c.VIVPAR_UT) AS sum_VIVPAR_UT,
    SUM(c.OCUPVIVPAR) AS sum_OCUPVIVPAR,
    SUM(c.PROM_OCUP) AS sum_PROM_OCUP,
    SUM(c.PRO_OCUP_C) AS sum_PRO_OCUP_C,
    SUM(c.VPH_PISODT) AS sum_VPH_PISODT,
    SUM(c.VPH_PISOTI) AS sum_VPH_PISOTI,
    SUM(c.VPH_1DOR) AS sum_VPH_1DOR,
    SUM(c.VPH_2YMASD) AS sum_VPH_2YMASD,
    SUM(c.VPH_1CUART) AS sum_VPH_1CUART,
    SUM(c.VPH_2CUART) AS sum_VPH_2CUART,
    SUM(c.VPH_3YMASC) AS sum_VPH_3YMASC,
    SUM(c.VPH_C_ELEC) AS sum_VPH_C_ELEC,
    SUM(c.VPH_S_ELEC) AS sum_VPH_S_ELEC,
    SUM(c.VPH_AGUADV) AS sum_VPH_AGUADV,
    SUM(c.VPH_AEASP) AS sum_VPH_AEASP,
    SUM(c.VPH_AGUAFV) AS sum_VPH_AGUAFV,
    SUM(c.VPH_TINACO) AS sum_VPH_TINACO,
    SUM(c.VPH_CISTER) AS sum_VPH_CISTER,
    SUM(c.VPH_EXCSA) AS sum_VPH_EXCSA,
    SUM(c.VPH_LETR) AS sum_VPH_LETR,
    SUM(c.VPH_DRENAJ) AS sum_VPH_DRENAJ,
    SUM(c.VPH_NODREN) AS sum_VPH_NODREN,
    SUM(c.VPH_C_SERV) AS sum_VPH_C_SERV,
    SUM(c.VPH_NDEAED) AS sum_VPH_NDEAED,
    SUM(c.VPH_DSADMA) AS sum_VPH_DSADMA,
    SUM(c.VPH_NDACMM) AS sum_VPH_NDACMM,
    SUM(c.VPH_SNBIEN) AS sum_VPH_SNBIEN,
    SUM(c.VPH_REFRI) AS sum_VPH_REFRI,
    SUM(c.VPH_LAVAD) AS sum_VPH_LAVAD,
    SUM(c.VPH_HMICRO) AS sum_VPH_HMICRO,
    SUM(c.VPH_AUTOM) AS sum_VPH_AUTOM,
    SUM(c.VPH_MOTO) AS sum_VPH_MOTO,
    SUM(c.VPH_BICI) AS sum_VPH_BICI,
    SUM(c.VPH_RADIO) AS sum_VPH_RADIO,
    SUM(c.VPH_TV) AS sum_VPH_TV,
    SUM(c.VPH_PC) AS sum_VPH_PC,
    SUM(c.VPH_TELEF) AS sum_VPH_TELEF,
    SUM(c.VPH_CEL) AS sum_VPH_CEL,
    SUM(c.VPH_INTER) AS sum_VPH_INTER,
    SUM(c.VPH_STVP) AS sum_VPH_STVP,
    SUM(c.VPH_SPMVPI) AS sum_VPH_SPMVPI,
    SUM(c.VPH_CVJ) AS sum_VPH_CVJ,
    SUM(c.VPH_SINRTV) AS sum_VPH_SINRTV,
    SUM(c.VPH_SINLTC) AS sum_VPH_SINLTC,
    SUM(c.VPH_SINCINT) AS sum_VPH_SINCINT,
    SUM(c.VPH_SINTIC) AS sum_VPH_SINTIC
FROM nacional_final_completa n
JOIN censo_geo_int_centroid_buffer_500 c
  ON ST_Intersects(n.geometry, c.buffer_500)
GROUP BY n.id;
""")
print("Agregaciones de censo para buffers de 100 m y 500 m creadas.")

# --- Paso 4: Unir las agregaciones de censo con la tabla nacional_final_completa ---
# Obtener la lista de columnas de la tabla censo_agg_100 (se espera que sean las mismas en censo_agg_500)
schema = conn.execute("PRAGMA table_info(censo_agg_100)").fetchall()
# Cada fila del schema tiene la forma: (cid, name, type, notnull, dflt_value, pk)
cols = [row[1] for row in schema if row[1].startswith("sum_")]

# Generar la lista de expresiones para censo_agg_100 (buffer 100)
col_list_100 = ",\n    ".join([f"COALESCE(a.{col}, 0) AS censo_{col[4:]}_100" for col in cols])
# Generar la lista de expresiones para censo_agg_500 (buffer 500)
col_list_500 = ",\n    ".join([f"COALESCE(b.{col}, 0) AS censo_{col[4:]}_500" for col in cols])

# Construir el query final usando f-string
query = f"""
DROP TABLE IF EXISTS nacional_final_completa_con_censo;
CREATE TABLE nacional_final_completa_con_censo AS
SELECT
    n.*,
    {col_list_100},
    {col_list_500}
FROM nacional_final_completa n
LEFT JOIN censo_agg_100 a ON n.id = a.national_id
LEFT JOIN censo_agg_500 b ON n.id = b.national_id;
"""

# Ejecutar el query
conn.execute(query)

print("Tabla 'nacional_final_completa_con_censo' creada con las variables agregadas del censo.")

conn.close()


In [ ]:
# Conectar a la base de datos
db_path = "/content/denue_total_est_per_ocu_final_5.duckdb"
conn = duckdb.connect(db_path)

# Ejecutar PRAGMA table_info para obtener la estructura de la tabla
fields = conn.execute("PRAGMA table_info(nacional_final_completa_con_censo)").fetchall()

print("Lista de campos de 'nacional_final_completa_con_censo':")
for field in fields:
    # Cada 'field' es una tupla: (cid, name, type, notnull, dflt_value, pk)
    print(f"- {field[1]} ({field[2]})")

conn.close()


In [ ]:
# Conectar a la base de datos y extraer los datos con la geometría en WKB
conn = duckdb.connect('/content/denue_total_est_per_ocu_final_5.duckdb')
conn.execute("INSTALL spatial; LOAD spatial;")

df = conn.execute("""
    SELECT
        *,
        ST_AsWKB(geometry) AS geom_wkb
    FROM nacional_final_completa_con_censo
""").df()
conn.close()

# Función segura para cargar la geometría a partir de WKB
def safe_wkb_load(x):
    # Verificar si x es del tipo adecuado (bytes o bytearray)
    if isinstance(x, (bytes, bytearray)):
        try:
            return wkb.loads(x)
        except Exception as e:
            return None
    else:
        return None

# Aplicar la función a la columna 'geom_wkb'
df['geom'] = df['geom_wkb'].apply(safe_wkb_load)

# Eliminar las columnas originales 'geometry' y 'geom_wkb'
df = df.drop(columns=['geometry', 'geom_wkb'])

# Crear el GeoDataFrame usando la columna 'geom' como geometría activa
gdf = gpd.GeoDataFrame(df, geometry='geom', crs="EPSG:4326")

# Exportar a GeoPackage
gdf.to_file("nacional_final_completa_censo.gpkg", driver="GPKG")

print("GeoPackage 'nacional_final_completa_censo.gpkg' creado exitosamente.")

In [ ]:
gdf = gpd.read_file("nacional_final_completa_censo.gpkg")

In [ ]:
gdf

In [ ]:
gdf["epo_100"] =  gdf["epo_100"] - gdf["personal"]
gdf["epo_500"] =  gdf["epo_500"] - gdf["personal"]
gdf["act_46_100"] =  gdf["act_46_100"] - gdf["personal"]
gdf["act_46_500"] =  gdf["act_46_500"] - gdf["personal"]

In [ ]:
X = gdf.drop(columns=['klass', 'id', 'codigo_act','codigo_act_2c','clee','nom_estab','cve_ent','personal','geometry']) #geom la primera vez jeje
y = gdf["klass"]

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [ ]:
exported_pipeline = GradientBoostingClassifier(
    learning_rate=0.05, max_depth=3, min_samples_leaf=14,
    min_samples_split=7, n_estimators=100, subsample=0.8
)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Suponiendo que ya has entrenado el modelo:
# exported_pipeline.fit(X_train, y_train)
# results = exported_pipeline.predict(X_test)

# Exactitud (accuracy)
acc = accuracy_score(y_test, results)
print(f"Accuracy: {acc:.4f}")

# Reporte de clasificación
report = classification_report(y_test, results)
print("Classification Report:")
print(report)

# Matriz de confusión
cm = confusion_matrix(y_test, results)
print("Confusion Matrix:")
print(cm)

# (Opcional) Visualizar la matriz de confusión con seaborn
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import joblib

In [ ]:

# Guardar el modelo entrenado
joblib.dump(exported_pipeline, 'trained_model.joblib')

In [ ]:
# denue_total_est_per_ocu_final_5.duckdb
# datos_censo_nacional_s9.duckdb

In [ ]:
from google.colab import drive
import shutil
import os

# Crear directorio destino en Drive si no existe
dest_dir = '/content/drive/MyDrive/COLMEX ML'

# Lista de archivos a copiar
archivos = [
    '/content/denue_total_est_per_ocu_final_5.duckdb',
    '/content/datos_censo_nacional_s9.duckdb'
]

# Copiar cada archivo
for archivo in archivos:
    try:
        shutil.copy(archivo, dest_dir)
        print(f"✅ {os.path.basename(archivo)} copiado exitosamente a Drive")
    except FileNotFoundError:
        print(f"❌ Error: {archivo} no encontrado en Colab")
    except Exception as e:
        print(f"❌ Error inesperado copiando {archivo}: {str(e)}")

# Verificar resultado final
print("\nArchivos en tu Drive después de la copia:")
print(os.listdir(dest_dir))